In [ ]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
import sys
if ".." not in sys.path:
    sys.path.append("..")
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# TD_08_3 - Le langage SQL 2/2

On peut aussi utiliser la base de données présente localement en utilisant la bibliothèque **sqlite3** de python. La requête SQL est alors envoyée sous forme d'une chaine de caractères

In [4]:
import sqlite3
connection = sqlite3.connect("ipt.sqlite3")
cursor = connection.cursor()

La base **ipt.sqlite3** contient 6 tables dont la table **album**

In [5]:
# Affichage des différentes relations disponibles
for row in cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';"""):
    print(row)

('album',)
('bbc',)
('cia',)
('style',)
('totp',)
('track',)


In [6]:
for row in cursor.execute("PRAGMA table_info(album);"):
    print(row)

(0, 'asin', 'char(10)', 1, None, 0)
(1, 'title', 'varchar(50)', 0, 'NULL', 0)
(2, 'artist', 'varchar(50)', 0, 'NULL', 0)
(3, 'price', 'decimal(10,2)', 0, 'NULL', 0)
(4, 'rdate', 'date', 0, 'NULL', 0)
(5, 'label', 'varchar(50)', 0, 'NULL', 0)
(6, 'rank', 'integer', 0, 'NULL', 0)


Les donnees sont issues d’amazon. L’attribut **price** représente le prix en dollars, l’attribut **rank** est le rang dans les ventes de disques. L'**ASIN** (Amazon Standard Identification Number) est un bloc unique de 10 lettres et/ou chiffres qui identifient les articles. L'ASIN figure sur la page d'informations du produit sur le site Amazon.fr.

## Exercice 1
1. Chercher tous les albums de U2
2. Chercher tous les albums parus en 1991

In [7]:
for row in cursor.execute("""
SELECT title FROM album
WHERE artist = 'U2';
"""):
    print(row)

('Achtung Baby',)
('Joshua Tree',)


In [8]:
for row in cursor.execute("""
SELECT title,rdate FROM album
WHERE rdate LIKE '1991%';
"""):
    print(row)

('Achtung Baby', '1991-11-19')
('Ten', '1991-08-27')
('Metallica', '1991-08-12')
('Mack the Knife', '1991-11-12')
('Eurythmics - Greatest Hits', '1991-05-24')
('Nevermind', '1991-09-24')


On peut utiliser la requête agrégative **COUNT** et le mot clé **GROUP BY** pour indiquer quelles lignes doivent être regroupées. Il existe aussi **AVG** (average), **MAX**, **MIN** et **SUM**. Pour filtrer selon une condition après avoir regroupé les lignes, on utilise le mot clé **HAVING**, le mot clé **WHERE** est utilisé pour filtrer les lignes avant les avoir regroupées.

## Exercice 2
3. Qui a le plus d’albums dans les 10000 meilleures ventes le label Sony ou le label Atlantic ?
4. Combien d’artistes ont un album dans les 5000 meilleures ventes ?

In [9]:
for row in cursor.execute("""
SELECT COUNT(title) AS nbr,label FROM album
WHERE (label = 'Sony' OR label = "Atlantic") AND rank < 10000 GROUP BY label ORDER BY nbr DESC;
"""):
    print(row)

(26, 'Sony')
(5, 'Atlantic')


In [10]:
for row in cursor.execute("""
SELECT COUNT(DISTINCT artist)""") #NOT FINISHEDD

SyntaxError: invalid syntax (<ipython-input-10-d4bab7b2bb80>, line 2)

On considérera aussi la table **track**

In [11]:
for row in cursor.execute("PRAGMA table_info(track);"):
    print(row)

(0, 'album', 'char(10)', 1, None, 0)
(1, 'disk', 'integer', 1, None, 0)
(2, 'posn', 'integer', 1, None, 0)
(3, 'song', 'varchar(255)', 0, 'NULL', 0)


L’attribut **album** de cette table reprend l’attribut **asin** de la table précédente pour identifier chaque album.

Il est possible de filtrer en utilisant un masque à l'aide du mot clé **LIKE**. Le masque peut contenir les caractères % qui représente n'importe quelle chaine de caractère (y compris une chaine vide) et _ qui représente n'importe quel caractère (un seul). WHERE name LIKE '%e%e%' va conserver tous les noms contenant 2 e par exemple.

## Exercice 3
5. Chercher tous les asin des albums parus en 1991. En déduire la liste de tous les titres de chansons dans des albums parus en 1991.
6. Chercher l’ensemble des **song**, **title**, **artist** pour les chansons commençant par S d’artistes commençant par B 


In [12]:
#6
for row in cursor.execute("""
SELECT track.song, album.title, album.artist
FROM album JOIN track ON album.asin = track.album
WHERE track.song LIKE 'S%' AND album.artist LIKE 'B%'"""):
    print(row)

('Song 2', 'Blur', 'Blur')
('Strange News From Another Star', 'Blur', 'Blur')
('Salsation - David Shire', 'Saturday Night FeverThe Original Movie Sound Track', 'Bee Gees')
("Stayin' Alive - Bee Gees", 'Saturday Night FeverThe Original Movie Sound Track', 'Bee Gees')
('Satisfy My Soul', 'Legend', 'Bob Marley')
('Stir It Up', 'Legend', 'Bob Marley')
('Sam, You Made the Pants Too Long', 'Barbra Streisand - Greatest Hits', 'Barbara Streisand')
('Second Hand Rose', 'Barbra Streisand - Greatest Hits', 'Barbara Streisand')
('Shelter from the Storm', 'Blood on the Tracks', 'Bob Dylan')
('Simple Twist of Fate', 'Blood on the Tracks', 'Bob Dylan')
("She's The One", 'Born to Run', 'Bruce Springsteen')
('Someone That I Used to Love [#]', 'Greatest Hits...and More', 'Barbara Streisand')
('Somewhere [From West Side Story]', 'Greatest Hits...and More', 'Barbara Streisand')
('Sand in My Shoes', '50 by Bobby Short', 'Bobby Short and his Orchestra')
('Simon Smith & His Dancing Bear', '50 by Bobby Short'

In [13]:
#5
for row in cursor.execute("""
SELECT song FROM track WHERE album IN (SELECT album.asin FROM album WHERE rdate LIKE '1991%');
"""):
    print(row)

('Zoo Station',)
('Even Better Than The Real Thing',)
('One',)
('Until The End Of The World',)
("Who's Gonna Ride Your Wild Horses",)
('So Cruel',)
('The Fly',)
('Mysterious Ways',)
("Tryin' To Throw Your Arms Around The World",)
('Ultra Violet (Light My Way)',)
('Acrobat',)
('Love Is Blindness',)
('Once',)
('Even Flow',)
('Alive',)
('Why Go',)
('Black',)
('Jeremy',)
('Oceans',)
('Porch',)
('Garden',)
('Deep',)
('Release',)
('Enter Sandman',)
('Sad but True',)
('Holier Than Thou',)
('Unforgiven',)
('Wherever I May Roam',)
("Don't Tread on Me",)
('Through the Never',)
('Nothing Else Matters',)
('Of Wolf and Man',)
('God That Failed',)
('My Friend of Misery',)
('Struggle Within',)
('Mack the Knife',)
('Lazy River',)
("That's the Way Love Is",)
('Beyond the Sea',)
('Was There a Call for Me',)
("I Guess I'm Good for Nothing But the Blues",)
("Don't Dream of Anybody But Me",)
('Guys and Dolls',)
('Down With Love',)
('Black Coffee',)
("Pete Kelly's Blues",)
('Clementine',)
("Bill Bailey, Won

In [14]:
connection = sqlite3.connect("ipt3.sqlite3")
cursor = connection.cursor()

La base **ipt3.sqlite3** contient 3 tables:
* actor
* casting
* movie

In [15]:
# Affichage des différentes relations disponibles
for row in cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';"""):
    print(row)

('actor',)
('casting',)
('movie',)


In [16]:
for row in cursor.execute("PRAGMA table_info(actor);"):
    print(row)

(0, 'id', 'integer', 1, None, 0)
(1, 'name', 'varchar(35)', 0, 'NULL', 0)


Dans la table **actor**, l'attribut **id** permet d'identifier un acteur de manière unique (clé primaire) l'associe à son nom **name**. Cette table contient également les réalisateurs.

In [17]:
for row in cursor.execute("PRAGMA table_info(movie);"):
    print(row)

(0, 'id', 'integer', 1, None, 0)
(1, 'title', 'varchar(70)', 0, 'NULL', 0)
(2, 'yr', 'decimal(4,0)', 0, 'NULL', 0)
(3, 'score', 'float', 0, 'NULL', 0)
(4, 'votes', 'integer', 0, 'NULL', 0)
(5, 'director', 'integer', 0, 'NULL', 0)


La table **movie** contient un attribut **id** (clé primaire) permettant d'identifier de manière unique chaque ligne. Cet id est associé au titre du film **title**, à l'année **yr** sous forme d'un nombre à 4 chiffres, à son réalisator **director**. L'attribut director fait référence à un id de la table actor. Un pur réalisateur peut ne pas être un vrai acteur mais sera présent dans la table actor quand même.


In [18]:
for row in cursor.execute("PRAGMA table_info(casting);"):
    print(row)

(0, 'movieid', 'integer', 1, None, 0)
(1, 'actorid', 'integer', 1, None, 0)
(2, 'ord', 'integer', 0, 'NULL', 0)


La table casting permet de mettre en relation la table actor et la table movie. L'entier **ord** indique l'ordre dans lequel sont énumérés les acteurs dans le casting d'un film : du premier nom de valeur 1 (le plus important) au dernier.

## Exercice 4
7. Déterminez le nom de tous les films qui commencent par « star » (16 réponses)
8. Déterminez le nombre d’acteurs pour chacun de ces films (sans les compter à la main:) )
9. Déterminez le nombre maximum d’acteurs au casting dans un film de cette base.
10. Déterminez les identifiants d’acteurs qui n’ont joué qu’en étant premier au casting
11. Déterminez pour chacun d’eux combien de films ils ont tourné
12. Déterminez le nom de tous les acteurs n’ayant joué qu’en étant premier au casting et ayant tourné plus de 5 films.
13. Ecrire une requete permettant d’afficher la liste des noms des réalisateurs (sans doublon)
14. Déterminer les noms des acteurs qui n’ont tourné qu’avec un seul réalisateur.
15. Déterminer les noms des acteurs qui n’ont tourné que dans des films qu’ils ont réalisés.

In [19]:
for row in cursor.execute("""
SELECT title FROM movie
WHERE title LIKE 'star%';
"""):
    print(row)

('Star Wars',)
('Star Wars: Episode V - The Empire Strikes Back',)
('Star Wars: Episode VI - Return of the Jedi',)
('Star Wars: Episode I - The Phantom Menace',)
('Star Trek: First Contact',)
('Starship Troopers',)
('Star Trek: The Wrath of Khan',)
('Stargate',)
('Star Trek: Insurrection',)
('Star Trek: Generations',)
('Star Trek IV: The Voyage Home',)
('Star Trek: The Motion Picture',)
('Star Trek VI: The Undiscovered Country',)
('Star Trek III: The Search for Spock',)
('Star Trek V: The Final Frontier',)
('Starman',)


In [28]:
for row in cursor.execute("""
SELECT title, COUNT(actorid) FROM movie JOIN casting ON movie.id = casting.movieid
WHERE title LIKE 'star%' GROUP BY title ORDER BY title;
"""):
    print(row)


('Star Trek III: The Search for Spock', 11)
('Star Trek IV: The Voyage Home', 11)
('Star Trek V: The Final Frontier', 9)
('Star Trek VI: The Undiscovered Country', 13)
('Star Trek: First Contact', 11)
('Star Trek: Generations', 15)
('Star Trek: Insurrection', 12)
('Star Trek: The Motion Picture', 11)
('Star Trek: The Wrath of Khan', 12)
('Star Wars', 10)
('Star Wars: Episode I - The Phantom Menace', 12)
('Star Wars: Episode V - The Empire Strikes Back', 11)
('Star Wars: Episode VI - Return of the Jedi', 13)
('Stargate', 10)
('Starman', 8)
('Starship Troopers', 9)


In [30]:
for row in cursor.execute("""
SELECT title, COUNT(actorid) as nbr FROM movie JOIN casting ON movie.id = casting.movieid
GROUP BY title ORDER BY nbr DESC LIMIT 1;
"""):
    print(row)


('Three Musketeers, The', 24)


In [31]:
#13
for row in cursor.execute("""
SELECT actor.name FROM movie JOIN actor ON movie.director = actor.id
WHERE ;
"""):
    print(row)


(360,)
(1040,)
(444,)
(198,)
(651,)
(639,)
(18,)
(18,)
(22,)
(2542,)
(18,)
(28,)
(1020,)
(110,)
(281,)
(67,)
(360,)
(966,)
(290,)
(198,)
(414,)
(111,)
(82,)
(198,)
(86,)
(67,)
(675,)
(408,)
(271,)
(389,)
(59,)
(110,)
(414,)
(82,)
(110,)
(37,)
(18,)
(291,)
(444,)
(253,)
(82,)
(196,)
(78,)
(133,)
(28,)
(198,)
(574,)
(86,)
(59,)
(597,)
(678,)
(1011,)
(21,)
(11,)
(67,)
(2459,)
(1044,)
(678,)
(2248,)
(38,)
(28,)
(204,)
(82,)
(408,)
(78,)
(18,)
(62,)
(500,)
(82,)
(1485,)
(445,)
(1151,)
(1040,)
(820,)
(604,)
(144,)
(201,)
(108,)
(31,)
(1106,)
(621,)
(2663,)
(107,)
(421,)
(421,)
(148,)
(11,)
(82,)
(59,)
(111,)
(198,)
(639,)
(2757,)
(11,)
(445,)
(196,)
(11,)
(445,)
(204,)
(2247,)
(2859,)
(604,)
(18,)
(37,)
(1151,)
(37,)
(105,)
(109,)
(10,)
(148,)
(270,)
(1070,)
(69,)
(666,)
(1103,)
(2043,)
(140,)
(2,)
(144,)
(993,)
(2586,)
(31,)
(47,)
(686,)
(414,)
(290,)
(1037,)
(198,)
(1069,)
(201,)
(2103,)
(2673,)
(18,)
(37,)
(264,)
(674,)
(133,)
(2118,)
(589,)
(1114,)
(383,)
(69,)
(298,)
(107,)
(2539,)
(104